<center><h1>Sentiment Analysis using TextBlob's Pretrained Model</h1></center>
<br>
<center>Sentiment analysis refers to the use of natural language processing, text analysis, computational linguistics, and biometrics to systematically identify, extract, quantify, and study affective states and subjective information.</center>
<br>
<center><img heigh=300 width=300 src=https://textblob.readthedocs.io/en/dev/_static/textblob-logo.png></center>
<center><h4>I will be using the TextBlob Python package for predicting subjectivity and polarity of the definitions of these Urban Dictionary words; I will compare the polarity to up_vote ratings to see if there is a correlation between positivity and rating or other such correlations. I'm a beginner with this so please feel free to comment and yell at me how bad this is and then maybe how to improve.</h4></center>


In [ ]:
# data processing
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# graphing
import plotly.express as px
import seaborn as sns # used for plotting correlation graph
import matplotlib.pyplot as plt

from textblob import TextBlob

# Quick Data Visualization

In [ ]:
df = pd.read_csv('/kaggle/input/urban-dictionary-words-dataset/urbandict-word-defs.csv', error_bad_lines=False)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.isna().sum()

In [ ]:
df = df.dropna()
df.isna().sum()

In [ ]:
ds = df[['word', 'up_votes', 'down_votes', 'definition']].copy()

In [ ]:
ds.loc['up_votes'] = pd.to_numeric(ds['up_votes'], errors='coerce')
ds.loc['down_votes'] = pd.to_numeric(ds['down_votes'], errors='coerce')
ds = ds.dropna()

In [ ]:
ds.head()

In [ ]:
ds[ds.down_votes.isin([0, 0.0])] = 1
ds[ds.up_votes.isin([0, 0.0])] = 1
ds = ds[ds.down_votes > 0]
ds = ds[ds.up_votes > 0]
ds['ratio_of_votes'] = ds.up_votes.div(ds.down_votes)
ds['total_votes'] = ds.up_votes.add(ds.down_votes)

In [ ]:
ds.sort_values(by=['ratio_of_votes'], ascending=False).tail(10)

In [ ]:
ds = ds.sort_values(['ratio_of_votes'])
fig = px.bar(
    ds.tail(40), 
    x="ratio_of_votes", 
    y="word", 
    text='definition',
    orientation='h', 
    title='Best ratio of votes', 
    width=2000, 
    height=800
)
fig.update_traces(textposition='outside')
fig.show()

In [ ]:
ds = ds.sort_values(['ratio_of_votes'], ascending=False)
fig = px.bar(
    ds.tail(40), 
    x="ratio_of_votes", 
    y="word", 
    text='definition',
    orientation='h', 
    title='Worst ratio of votes', 
    width=2000, 
    height=800
)
fig.update_traces(textposition='outside')
fig.show()

In [ ]:
ds1 = df['author'].value_counts().reset_index()
ds1.columns = ['author', 'count']
ds1 = ds1.sort_values(['count'])
fig = px.bar(
    ds1.tail(40), 
    x="count", 
    y="author", 
    orientation='h', 
    title='Top 40 authors by number of publications (authors are just hashes of the name :( )', 
    width=800, 
    height=800
)
fig.show()

# SENTIMENT ANALYSIS:

In [ ]:
ds['definition'] = ds['definition'].apply(str)

In [ ]:
ds.head()

In [ ]:
polarity = []
subjectivity = []
for i in range(ds.shape[0]):
    blob = TextBlob(ds['definition'].values[i])
    sentiment = blob.sentiment
    polarity.append(sentiment[0])
    subjectivity.append(sentiment[1])

In [ ]:
ds['polarity'] = polarity
ds['subjectivity'] = subjectivity

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, sharey=False, figsize=(16, 8))
plt.style.use('seaborn-pastel')
ax1.boxplot(ds.polarity)
ax1.set_title('Polarity')
ax2.set_title('Subjectivity')
ax2.boxplot(ds.subjectivity);

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True, figsize=(16, 20))
ax1.hist(ds.polarity, bins=20, density=True, color='skyblue')
ax1.set_title("Histogram of Polarity")
ax2.hist(ds.polarity[ds.ratio_of_votes >15], bins=20, color='lightgreen', density=True)
ax2.set_title("Polarity with ratio of votes > 15")
ax3.hist(ds.polarity[ds.ratio_of_votes > 50], bins=20, color='lightpink', density=True)
ax3.set_title("Polarity with ratio of votes > 50");

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True, figsize=(16, 20))
ax1.hist(ds.subjectivity, bins=20, density=True, color='skyblue')
ax1.set_title("Subjectivity")
ax2.hist(ds.subjectivity[ds.ratio_of_votes >15], bins=20, color='lightgreen', density=True)
ax2.set_title("Subjectivity with ratio of votes > 15")
ax3.hist(ds.subjectivity[ds.ratio_of_votes > 50], bins=20, color='lightpink', density=True)
ax3.set_title("Subjectivity with ratio of votes > 50");

In [ ]:
# words, definitions that obtained completely positive sentiment values
print(ds.loc[ds.polarity == 1, 'definition'].head(10))
print(ds.loc[ds.polarity == 1, 'word'].head(10))

## Correlation
maybe there is a correlation between positivity (high polarity) and up_votes or down_votes

In [ ]:
plt.rcParams['figure.figsize'] = (16, 8)
plt.style.use('fast')

sns.set(style="whitegrid")
corr = ds.corr()
sns.heatmap(corr,annot=True,cmap="coolwarm")

### **Conclusion**: there is no apparent non-trivial correlation